In [1]:
import requests
import json
import spacy

Load NLP file

In [2]:
nlp = spacy.load("./Spacy_fine_tuned_NER_for_Food")

### Google Speech API

In [ ]:
%pip install pyttsx3
%pip install pyaudio
%pip install py3-tts
%pip install speechrecognition

%brew install portaudio
%pip install pyaudio
# %pip install keyboard

In [71]:
import speech_recognition as sr
import pyttsx3 
# import keyboard

# sentence
sentence = []

# Initialize the recognizer 
r = sr.Recognizer() 

# Function to convert text to speech
def SpeakText(command):
	
	# Initialize the engine
	engine = pyttsx3.init("dummy")
	engine.say(command) 
	engine.runAndWait()
	

# Loop infinitely for user to speak
while True:
	# Exception handling to handle
	# exceptions at the runtime
	# if keyboard.is_pressed('enter'):
	# 	print("Pressed enter")
	# 	break

	try:
		# use the microphone as source for input.
		with sr.Microphone() as source2:
			# wait for a second to let the recognizer
			# adjust the energy threshold based on
			# the surrounding noise level 
			r.adjust_for_ambient_noise(source2, duration=0.2)
			
			#listens for the user's input 
			audio2 = r.listen(source2)
			
			# Using google to recognize audio
			MyText = r.recognize_google(audio2)
			MyText = MyText.lower()

			print(MyText)
			sentence.append(MyText)
			SpeakText(MyText)
			
	except sr.RequestError as e:
		print("Could not request results; {0}".format(e))
		
	except sr.UnknownValueError:
		print("unknown error occurred")


i hate blueberries cheesecake


KeyboardInterrupt: 

In [ ]:
print(sentence)

### NER food named entry recognition

In [77]:
# text = "I had pineapple fried rice with lime based highball."
# text = "I love barbecued chicken with corn salad with vanila ice cream."
text = sentence[0]

queries = [] # food names from the text

# Process the text using the loaded model
doc = nlp(text)

for ent in doc.ents:
    print(ent.text, ent.label_)
    queries.append(ent.text)

blueberries cheesecake FOOD


In [78]:
queries

['blueberries cheesecake']

In [63]:
app_id = "d5fd616b"
app_key = "5c06fb4f5bd545920f9d620b5a5a63d4"

Autocomplete the given query to find the food name.

In [79]:
queries_for_nutrients = [] # queries for finding the calories

for query in queries:
    body = {
        "query": query
    }

    url = "https://trackapi.nutritionix.com/v2/search/instant/?query=" + query
    headers = {
        "Content-Type": "application/json",
        "x-app-id": app_id,
        "x-app-key": app_key
    }

    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        raise Exception(f"Error: {response.status_code}")
    else:
        response_json = response.json()
        branded_food = 0   # check if we found the same food name as query in common food, if not, we use the first element of branded foods
        
        if "common" in response_json:
            for common in response_json["common"]:
                if common["food_name"].lower() == query.lower(): # if there's a name matching
                    branded_food = 1
                    print(common["food_name"])
                    queries_for_nutrients.append(common["food_name"])
                    break

        if branded_food == 0 and "branded" in response_json:
            if response_json["branded"]:
                print(response_json["branded"][0]["food_name"]) # pop the first element of branded food
                queries_for_nutrients.append(response_json["branded"][0]["food_name"])


Buttermilk Pancakes with Wild Blueberries


In [80]:
print(queries_for_nutrients)

['Buttermilk Pancakes with Wild Blueberries']


Since we found the food name, print the calories.

In [81]:
calories_information = [] # have {query, calories}
index = 0
total_calories = 0

url = "https://trackapi.nutritionix.com/v2/natural/nutrients"
headers = {
    "Content-Type": "application/json",
    "x-app-id": app_id,
    "x-app-key": app_key
}
for query_for_nutrients in queries_for_nutrients:
    body = {
        "query": query_for_nutrients
    }

    response = requests.post(url, headers=headers, data=json.dumps(body))

    # check if the key "foods" exists in the JSON response
    if "foods" in response.json():
        sum_calories = 0
        for food in response.json()["foods"]:
            print(food["food_name"] + ": " + str(food["nf_calories"]) + " calories")
            sum_calories += food["nf_calories"]

        total_calories += sum_calories
        calories_information.append({queries[index]: sum_calories})
        index = index + 1
    else:
        print("No foods found for query:", query)

buttermilk pancakes: 174.79 calories
wild blueberries: 39.9 calories


In [82]:
print(calories_information)

[{'blueberries cheesecake': 214.69}]


In [83]:
print(total_calories)

214.69
